In [1]:
import os
import re
import pandas as pd
import numpy
import time
from gensim.models.fasttext import FastText
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import neighbors
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import nltk
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


from nltk.corpus import stopwords
import multiprocessing

cores = multiprocessing.cpu_count()
FastTextModel = "fasttext_model.sav"

allContent = ""
path1 = "../input/ps-dataset/powershell_benign_dataset/"
path2 = "../input/ps-dataset/malicious_pure"
#path3 = r"D:\Test Scripts"
labels = []
text = []
testText = []
#new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
fName= []
newText = []
regex = r':\s*\n+'
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

def test_bowser(path):
        with open(path, encoding='utf-8') as f:
            t_content = f.read()
            t_content = t_content.lower()
            t_content = t_content.replace("(", "")
            t_content = t_content.replace(")", "")
            t_content = t_content.replace("[", "")
            t_content = t_content.replace("]", "")
            t_content = t_content.replace("$", "")
            t_content = t_content.replace("-", "")
            t_content = t_content.replace("=", "")
            t_content = t_content.replace("", "")
            t_content = t_content.replace("{", "")
            t_content = t_content.replace("}", "")
            t_content = t_content.replace(".", "")
            t_content = t_content.replace("_", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace(":", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace("/", "")
            t_content = t_content.replace("@", "")
            t_content = t_content.replace("?", "")
            t_content = t_content.replace(",", "")
            t_content = t_content.replace(">", "")
            t_content = t_content.replace("<", "")
            t_content = t_content.replace(";", "")
            t_content = t_content.replace("\\", "")
            t_content = t_content.replace("+", "")
            t_content = t_content.replace("\"","")
            t_content = t_content.replace("\'","")
            t_content = t_content.replace("`", "")
            t_content = t_content.replace("*", "")
            t_content = " ".join(t_content.split())
            newText.append(t_content)
            return t_content

def bowser(path, label):
    for filename in os.listdir(path):
        labelName = "__label__" + label
        labels.append(labelName)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
                    content = f.read()
                    content = content.lower()
                    content = content.replace("(", "")
                    content = content.replace(")", "")
                    content = content.replace("[", "")
                    content = content.replace("]", "")
                    content = content.replace("$", "")
                    content = content.replace("-", "")
                    content = content.replace("!","")
                    content = content.replace("=", "")
                    content = content.replace("", "")
                    content = content.replace("{", "")
                    content = content.replace("}", "")
                    content = content.replace(".", "")
                    content = content.replace("_", "")
                    content = content.replace("|", "")
                    content = content.replace(":", "")
                    content = content.replace("|", "")
                    content = content.replace("/", "")
                    content = content.replace("@", "")
                    content = content.replace("?", "")
                    content = content.replace(",", "")
                    content = content.replace(">", "")
                    content = content.replace("<", "")
                    content = content.replace(";", "")
                    content = content.replace("\\", "")
                    content = content.replace("+", "")
                    content = content.replace("\"", "")
                    content = content.replace("\'", "")
                    content = content.replace("`", "")
                    content = content.replace("*", "")
                    content = " ".join(content.split())
                    testText.append(content)


#test_bowser(path3)
bowser(path1,"Benign")
bowser(path2,"Malicious")
#print(testText[6])
numFiles = len(testText)
#res = [re.sub(r"(\w)([A-Z])", r"\1 \2", ele) for ele in testText]
#print(res[1] + "\n")

df = pd.DataFrame(testText, columns=["Powershell Code"])
df['Labels'] = labels
#df.to_csv('tester.csv')
print(df)

train, test = train_test_split(df, test_size=0.3, random_state=42)
train_size = len(train['Powershell Code'])
train.reset_index()
train.reindex(index=range(0,5962))
print(train)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [2]:
i =0
output = ""
while (i<5961):
    output = output + (train['Labels'].iloc[i]+" "+train['Powershell Code'].iloc[i]+".\n")
    i = i+1

with open('powershell.txt', 'w') as f:
    f.write(output)

In [3]:
import fasttext
model = fasttext.train_supervised(input="./powershell.txt")
model.save_model("model_powershell.bin")



In [4]:
i=0
while (i<55):
    print("\n")
    input = test['Powershell Code'].iloc[i]
    print(model.predict(input))
    print("Correct Label:"+ test['Labels'].iloc[i])
    i=i+1

In [5]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

test_data = list(test['Powershell Code'])
test_labels = list(test['Labels'])
#print(model.predict(test_data)[0])
y_pred = model.predict(test_data)

#print(test_labels)
print(classification_report(test_labels,y_pred[0],digits=6))

In [6]:
st = time.time()
kk = test_bowser("../input/ps-dataset/malicious_pure/1010.ps1")
print(model.predict(kk))
end = time.time()

print(end-st)

## Dataset 2

In [9]:
dataset_2 = pd.read_csv('../input/ps-dataset-2/Dataset_2.csv')
p=0
while p < 6804:
    dataset_2['Labels'].iloc[p] = "__label__" + dataset_2['Labels'].iloc[p]
    p= p+1

train2, test2 = train_test_split(dataset_2, test_size=0.3, random_state=42)

i =0
output = ""
while (i<4763):
    output = output + "__label__"+train2['Labels'].iloc[i] +" "+ (train2['Powershell Code'].iloc[i])+".\n"
    i = i+1
    

with open('powershell_dataset2.txt', 'w') as f:
    f.write(output)


In [10]:
import fasttext
model2 = fasttext.train_supervised(input="../input/ps-fasttext-dataset-2/powershell_dataset2 (5).txt")
model2.save_model("model2_powershell.bin")



In [11]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

test_data2 = list(test2['Powershell Code'])
test_labels2 = list(test2['Labels'])
#print(model.predict(test_data)[0])
y_pred = model2.predict(test_data2)

#print(test_labels)
print(classification_report(test_labels2,y_pred[0],digits=6))

In [13]:
#CROSS VAL DATASET2 ON MODEL 1
y_pred = model.predict(test_data2)

#print(test_labels)
print(classification_report(test_labels2,y_pred[0],digits=6))

In [14]:
#CROSS VAL DATASET1 ON MODEL 2
y_pred = model2.predict(test_data)

#print(test_labels)
print(classification_report(test_labels,y_pred[0],digits=6))

plot_learning_curves(X_train2, Y_train2, X_test2, Y_test2, svclassifier2)
plt.show()